## Assignment 3 - Competitive Assignment
An explanation this assignment could be found in the .pdf explanation document

### Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [18]:
# Id : 209040757
# Name : Yarin Ben Baruch


In [19]:
# imports for reading and writing (input & output) files:
import pandas as pd
import os
import re

In [20]:
### add whatever imports you need
# YOUR CODE HERE

import warnings
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import Pipeline
from collections import Counter


warnings.simplefilter("ignore")
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# *** KNN
from sklearn.neighbors import KNeighborsClassifier
# *** Decision Tree; Random Forest
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
# MLP
from sklearn.neural_network import MLPClassifier
# Ensemble
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# *** Naive Bayes
from sklearn.naive_bayes import GaussianNB,MultinomialNB
# *** SVM classifier
from sklearn.svm import SVC


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [22]:
train_filename = '.' + os.sep + 'input' + os.sep + 'annotated_corpus_for_train.xlsx'
test_filename  = '.' + os.sep + 'input' + os.sep + 'corpus_for_test.xlsx'


df_train = pd.read_excel(train_filename, 'corpus', index_col=None, na_values=['NA'])
df_test  = pd.read_excel(test_filename,  'corpus', index_col=None, na_values=['NA'])


### Your implementation:
Write your code solution in the following code-cells

In [23]:
# Split the data
def train(X,y,rand):

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state= rand)
    return x_train, x_test, y_train, y_test



#Regular expression on dataframe 
def regOnDate(df):
    
    for i in df.index:
        
        df.iloc[i,0] = re.sub(r'\W',' ',str(df.iloc[i,0]))# החלפת תווים מיוחדים
        df.iloc[i,0] = re.sub(r'\d',' ',str(df.iloc[i,0]))# החלפה מספרים
        df.iloc[i,0] = re.sub(r'\s\w\s',' ',df.iloc[i,0]) # החלפה אות בודדת

        
    return df

#regular expression on dataframe for the test
def regOnDateTest(df):
    
    for i in df.index:
        
        df.iloc[i,1] = re.sub(r'\W',' ',str(df.iloc[i,1]))# החלפת תווים מיוחדים
        df.iloc[i,1] = re.sub(r'\d',' ',str(df.iloc[i,1]))# החלפה מספרים
        df.iloc[i,1] = re.sub(r'\s\w\s',' ',df.iloc[i,1]) # החלפה אות בודדת
        
    return df


# F1 - avg women and men
def f1Avg(y_test, y_pred):
    
    conf_mat = metrics.confusion_matrix(y_test, y_pred) #

    TP = conf_mat[0][0]
    FP = conf_mat[0][1]
    FN = conf_mat[1][0]
    TN = conf_mat[1][1]
    
    
    precision_m = TN / (TN + FN)
    recall_m = TN / (TN + FP)

    precision_f = TP / (TP + FP)
    recall_f = TP / (TP + FN)


    F1_f = 2 * (precision_f * recall_f) / (precision_f + recall_f)
    F1_m = 2 * (precision_m * recall_m) / (precision_m + recall_m)

    F1_avg = (F1_f + F1_m) / 2
    
    return F1_avg



# Common word
def Words(df,num):
    
    words = []
    
    for i in df.index:
        split = str(df.iloc[i, 0]).split() # Split the data
        words = words + split # Add to list

        
    counter = Counter(words)

    commonWords = [key for key, _ in counter.most_common(num)] 
    
    
    return commonWords # Return the words list .



In [24]:

df_train = regOnDate(df_train) 

X = df_train['story']
y = df_train['gender']


y = y.replace('m', 1)
y = y.replace('f', 0)


# ML - TESTING

In [25]:

x_train, x_test, y_train, y_test = train(X,y,0)

## Looking for the successful model


## RandomForest

In [26]:
text_clf_RF = Pipeline([
    ('vect', CountVectorizer(stop_words= Words(df_train,80) , ngram_range=(1,2), max_features=6000)),
    ('clf', RandomForestClassifier(random_state=1))])


text_clf_RF.fit(x_train, y_train)
predicted_train = text_clf_RF.predict(x_test)
f1Avg(y_test, predicted_train)

Pipeline(steps=[('vect',
                 CountVectorizer(max_features=6000, ngram_range=(1, 2),
                                 stop_words=['את', 'לא', 'של', 'על', 'היה',
                                             'עם', 'שלי', 'לי', 'אני', 'כל',
                                             'זה', 'לאחר', 'מאוד', 'הוא', 'מה',
                                             'אבל', 'גם', 'שאני', 'כי', 'הייתה',
                                             'כמה', 'אחד', 'יותר', 'לנו', 'יום',
                                             'אחרי', 'אותו', 'כבר', 'אותי',
                                             'שלא', ...])),
                ('clf', RandomForestClassifier(random_state=1))])

nan

## KNeighbors

In [27]:
text_clf_KNN = Pipeline([
    ('vect', TfidfVectorizer(max_features=7500)),
    ('clf', KNeighborsClassifier(n_neighbors= 5))])

text_clf_KNN.fit(x_train, y_train)
predicted_train = text_clf_KNN.predict(x_test)
f1Avg(y_test, predicted_train)

Pipeline(steps=[('vect', TfidfVectorizer(max_features=7500)),
                ('clf', KNeighborsClassifier())])

0.6137566137566137

## DecisionTree

In [28]:
text_clf_DT = Pipeline([
    ('vect', CountVectorizer(max_features=6000,stop_words= Words(df_train,33))),
    ('clf', DecisionTreeClassifier(random_state=40))])


text_clf_DT.fit(x_train, y_train)
predicted_train = text_clf_DT.predict(x_test)
f1Avg(y_test, predicted_train)



Pipeline(steps=[('vect',
                 CountVectorizer(max_features=6000,
                                 stop_words=['את', 'לא', 'של', 'על', 'היה',
                                             'עם', 'שלי', 'לי', 'אני', 'כל',
                                             'זה', 'לאחר', 'מאוד', 'הוא', 'מה',
                                             'אבל', 'גם', 'שאני', 'כי', 'הייתה',
                                             'כמה', 'אחד', 'יותר', 'לנו', 'יום',
                                             'אחרי', 'אותו', 'כבר', 'אותי',
                                             'שלא', ...])),
                ('clf', DecisionTreeClassifier(random_state=40))])

0.6137566137566137

## Naive Bayes

In [30]:
text_clf_NB = Pipeline([
    ('vect', CountVectorizer(max_features= 2000, ngram_range=(1,4))),
    ('clf', MultinomialNB())])

text_clf_NB.fit(x_train, y_train)
predicted_train = text_clf_NB.predict(x_test)
f1Avg(y_test, predicted_train)



Pipeline(steps=[('vect',
                 CountVectorizer(max_features=2000, ngram_range=(1, 4))),
                ('clf', MultinomialNB())])

0.7598684210526316

## SVM classifier

In [31]:
text_clf_SVC = Pipeline([
    ('vect', TfidfVectorizer(max_features=9000)),
    ('clf', SVC())])



text_clf_SVC.fit(x_train, y_train)
predicted_train = text_clf_SVC.predict(x_test)
f1Avg(y_test, predicted_train)

Pipeline(steps=[('vect', TfidfVectorizer(max_features=9000)), ('clf', SVC())])

nan

## MLP classifier

In [32]:
text_clf_MLP = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,6), max_features=6000)),
    ('clf', MLPClassifier(alpha=1, max_iter=1000))])


text_clf_MLP.fit(x_train, y_train)
predicted_train = text_clf_MLP.predict(x_test)
f1Avg(y_test, predicted_train)

Pipeline(steps=[('vect',
                 CountVectorizer(max_features=6000, ngram_range=(1, 6))),
                ('clf', MLPClassifier(alpha=1, max_iter=1000))])

0.6256410256410255

## AdaBoost classifier

In [33]:
text_clf_AdaBoost = Pipeline([
    ('vect', TfidfVectorizer(stop_words=Words(df_train,74),max_features=5500,ngram_range=(1,2))),
    ('clf', AdaBoostClassifier())])


text_clf_AdaBoost.fit(x_train, y_train)
predicted_train = text_clf_AdaBoost.predict(x_test)
f1Avg(y_test, predicted_train)



Pipeline(steps=[('vect',
                 TfidfVectorizer(max_features=5500, ngram_range=(1, 2),
                                 stop_words=['את', 'לא', 'של', 'על', 'היה',
                                             'עם', 'שלי', 'לי', 'אני', 'כל',
                                             'זה', 'לאחר', 'מאוד', 'הוא', 'מה',
                                             'אבל', 'גם', 'שאני', 'כי', 'הייתה',
                                             'כמה', 'אחד', 'יותר', 'לנו', 'יום',
                                             'אחרי', 'אותו', 'כבר', 'אותי',
                                             'שלא', ...])),
                ('clf', AdaBoostClassifier())])

0.6313131313131313

# Result df_test

## The model chosen: Naive Bayes
## Grade : 0.759

In [34]:
df_test = regOnDateTest(df_test) 

X_test = df_test['story']

predDfTest = text_clf_NB.predict(X_test) 

In [35]:
test = df_test.copy()

for i in range(0,len(test['test_example_id'])):
    test['test_example_id'][i] = predDfTest[i] 
    

    
test['test_example_id'] = test['test_example_id'].replace(1,'m')
test['test_example_id'] = test['test_example_id'].replace(0,'f')  


### Save output to csv
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [36]:
df_predicted = pd.DataFrame(columns = ['test_example_id','predicted_category'])

df_predicted['test_example_id'] = df_test['test_example_id']
df_predicted['predicted_category'] = test['test_example_id']

In [37]:
df_predicted.to_csv('classification_results.csv',index=False)